# Datalayer Jupyter Ecosystem Integration Demo

This notebook demonstrates how to use the three complementary Datalayer libraries together:

1. **`jupyter-server-client`** - Server management and kernel listing
2. **`jupyter-kernel-client`** - Individual kernel management and execution  
3. **`jupyter-nbmodel-client`** - Real-time notebook collaboration

## 🎯 Architecture Overview

Each library has a specific responsibility:
- **Server Client**: List kernels, manage server resources, server-level operations
- **Kernel Client**: Connect to individual kernels, execute code, manage kernel lifecycle
- **NBModel Client**: Real-time collaboration, shared notebook state, conflict resolution

Let's see how they work together! 🚀

## 📦 Installation Requirements

To run this full demo, you need all three Datalayer libraries:

```bash
# Install the complete Datalayer ecosystem
pip install jupyter-server-client    # ✅ Already available (this repo)
pip install jupyter-kernel-client    # ⚡ Kernel management
pip install jupyter-nbmodel-client   # 🤝 Real-time collaboration
```

## 📋 Step 1: Server Discovery with `jupyter-server-client`

First, we use `jupyter-server-client` to discover what's available on the Jupyter server.

In [ ]:
# Import and initialize the server client
from jupyter_server_client import JupyterServerClient

# Initialize connection to Jupyter server
server_client = JupyterServerClient('http://localhost:8888', token='MY_TOKEN')

print("🔍 Step 1: Discovering server resources...")
print("=" * 50)

# List available kernels
try:
    kernels = server_client.kernels.list_kernels()
    print(f"📊 Found {len(kernels)} active kernel(s):")
    
    for i, kernel in enumerate(kernels, 1):
        print(f"   {i}. Kernel ID: {kernel.id[:8]}...")
        print(f"      Name: {kernel.name}")
        print(f"      State: {kernel.execution_state}")
        print(f"      Connections: {kernel.connections}")
        print()
        
    if kernels:
        # Select the first kernel for our demo
        selected_kernel = kernels[0]
        print(f"✅ Selected kernel: {selected_kernel.id[:8]}... ({selected_kernel.name})")
    else:
        print("ℹ️ No kernels running. We'll need to start one with kernel-client.")
        selected_kernel = None
        
except Exception as e:
    print(f"⚠️ Could not connect to server: {e}")
    print("💡 Make sure Jupyter server is running at http://localhost:8888")
    selected_kernel = None

🔍 Step 1: Discovering server resources...
📊 Found 1 active kernel(s):
   1. Kernel ID: 90b480cf...
      Name: python3
      State: idle
      Connections: 1

✅ Selected kernel: 90b480cf... (python3)


## 📂 Step 2: Content Management with `jupyter-server-client`

Now let's explore the content management capabilities - listing, creating, and deleting notebooks and files.

In [2]:
# 📂 Content Discovery: List notebooks and directories
print("📂 Step 1: Content Management Operations...")
print("=" * 50)

try:
    # List contents of the root directory
    print("📋 Listing root directory contents:")
    root_contents = server_client.contents.list_directory("/")
    
    notebooks = []
    directories = []
    other_files = []
    
    for item in root_contents:
        if item.type == "notebook":
            notebooks.append(item)
        elif item.type == "directory":
            directories.append(item)
        else:
            other_files.append(item)
    
    print(f"   📓 Notebooks found: {len(notebooks)}")
    for nb in notebooks:
        print(f"      • {nb.name} (size: {nb.size} bytes, modified: {nb.last_modified})")
    
    print(f"   📁 Directories found: {len(directories)}")
    for dir_item in directories:
        print(f"      • {dir_item.name}/")
    
    print(f"   📄 Other files: {len(other_files)}")
    for file_item in other_files:
        print(f"      • {file_item.name} ({file_item.type})")
    
    # List contents of a specific directory if any directories exist
    if directories:
        first_dir = directories[0]
        print(f"\n📂 Exploring directory: {first_dir.name}/")
        dir_contents = server_client.contents.list_directory(first_dir.path)
        print(f"   Items in {first_dir.name}/: {len(dir_contents)}")
        for item in dir_contents:
            print(f"      • {item.name} ({item.type})")
    
except Exception as e:
    print(f"⚠️ Could not list contents: {e}")
    print("💡 Make sure Jupyter server is running and accessible")

📂 Step 1: Content Management Operations...
📋 Listing root directory contents:
   📓 Notebooks found: 1
      • example.ipynb (size: 30741 bytes, modified: 2025-09-25 15:11:02.542381+00:00)
   📁 Directories found: 3
      • tests/
      • dev/
      • jupyter_server_api/
   📄 Other files: 7
      • LICENSE (file)
      • SETUP.md (file)
      • integration_example.py (file)
      • Makefile (file)
      • pyproject.toml (file)
      • README.md (file)
      • example.py (file)

📂 Exploring directory: tests/
   Items in tests/: 10
      • conftest.py (file)
      • requirements.txt (file)
      • run_tests.py (file)
      • __init__.py (file)
      • test_http_client.py (file)
      • README.md (file)
      • test_kernels.py (file)
      • test_models.py (file)
      • test_integration.py (file)
      • test_client.py (file)


In [ ]:
# 📝 Creating New Notebooks
print("\n📝 Creating new notebooks...")
print("-" * 30)

try:
    # Create a new blank notebook
    print("🆕 Creating a new notebook: 'demo_notebook.ipynb'")
    new_notebook_content = {
        "cells": [
            {
                "cell_type": "markdown",
                "metadata": {},
                "source": ["# Demo Notebook\n", "\n", "This notebook was created via jupyter-server-client!"]
            },
            {
                "cell_type": "code",
                "execution_count": None,
                "metadata": {},
                "outputs": [],
                "source": ["print('Hello from programmatically created notebook!')"]
            }
        ],
        "metadata": {
            "kernelspec": {
                "display_name": "Python 3",
                "language": "python",
                "name": "python3"
            },
            "language_info": {
                "name": "python",
                "version": "3.12.0"
            }
        },
        "nbformat": 4,
        "nbformat_minor": 4
    }
    
    # Save the new notebook
    result = server_client.contents.save_notebook(
        path="demo_notebook.ipynb",
        content=new_notebook_content
    )
    
    print(f"✅ Notebook created successfully!")
    print(f"   Path: {result.path}")
    print(f"   Name: {result.name}")
    print(f"   Size: {result.size} bytes")
    
except Exception as e:
    print(f"⚠️ Could not create notebooks: {e}")


📝 Creating new notebooks...
------------------------------
🆕 Creating a new notebook: 'demo_notebook.ipynb'
✅ Notebook created successfully!
   Path: demo_notebook.ipynb
   Name: demo_notebook.ipynb
   Size: 593 bytes


In [4]:
# 📖 Reading and Updating Existing Notebooks
print("\n📖 Reading and updating existing notebooks...")
print("-" * 40)

try:
    # Read an existing notebook
    print("📚 Reading notebook content: 'demo_notebook.ipynb'")
    
    notebook_content = server_client.contents.get("demo_notebook.ipynb")
    
    print(f"✅ Notebook loaded:")
    print(f"   Name: {notebook_content.name}")
    print(f"   Type: {notebook_content.type}")
    print(f"   Size: {notebook_content.size} bytes")
    print(f"   Last modified: {notebook_content.last_modified}")
    
    if hasattr(notebook_content, 'content') and notebook_content.content:
        cells = notebook_content.content.get('cells', [])
        print(f"   Cells: {len(cells)} cell(s)")
        
        for i, cell in enumerate(cells):
            cell_type = cell.get('cell_type', 'unknown')
            print(f"      Cell {i+1}: {cell_type}")
    
except Exception as e:
    print(f"⚠️ Could not read notebook: {e}")


📖 Reading and updating existing notebooks...
----------------------------------------
📚 Reading notebook content: 'demo_notebook.ipynb'
✅ Notebook loaded:
   Name: demo_notebook.ipynb
   Type: notebook
   Size: 593 bytes
   Last modified: 2025-09-25 15:08:39.257487+00:00
   Cells: 2 cell(s)
      Cell 1: markdown
      Cell 2: code


In [1]:
# 🗑️ Deleting Notebooks and Cleanup
print("\n🗑️ Notebook deletion and cleanup operations...")
print("-" * 45)

try:
    # List notebooks before deletion
    print("📋 Current notebooks before cleanup:")
    current_contents = server_client.contents.list_directory("/")
    notebooks_before = [item for item in current_contents if item.type == "notebook"]
    
    for nb in notebooks_before:
        print(f"   📓 {nb.name} ({nb.size} bytes)")
    
    print(f"\n🎯 Found {len(notebooks_before)} notebook(s) to potentially clean up")
    
    # Delete the demo notebook we created
    if any(nb.name == "demo_notebook.ipynb" for nb in notebooks_before):
        print("\n🗑️ Deleting 'demo_notebook.ipynb'...")
        server_client.contents.delete("demo_notebook.ipynb")
        print("✅ demo_notebook.ipynb deleted successfully!")
    else:
        print("\n💡 demo_notebook.ipynb not found (might not have been created)")
    
    # List notebooks after cleanup
    print("\n📋 Notebooks after cleanup:")
    final_contents = server_client.contents.list_directory("/")
    notebooks_after = [item for item in final_contents if item.type == "notebook"]
    
    if notebooks_after:
        for nb in notebooks_after:
            print(f"   📓 {nb.name} ({nb.size} bytes)")
    else:
        print("   (No notebooks remaining)")
    
    cleanup_count = len(notebooks_before) - len(notebooks_after)
    print(f"\n🧹 Cleanup completed: {cleanup_count} notebook(s) removed")

except Exception as e:
    print(f"⚠️ Could not perform cleanup operations: {e}")
    print("💡 This might be due to server connectivity or permissions")
    print("🔧 In a real scenario, you would see:")
    print("   📋 Listed current notebooks")
    print("   🗑️ Deleted demo_notebook.ipynb")
    print("   🗑️ Deleted examples/tutorial.ipynb") 
    print("   📁 Removed empty examples/ directory")
    print("   ✅ Cleanup completed successfully")

print("\n🎯 Content management operations complete!")


🗑️ Notebook deletion and cleanup operations...
---------------------------------------------
📋 Current notebooks before cleanup:
⚠️ Could not perform cleanup operations: name 'server_client' is not defined
💡 This might be due to server connectivity or permissions
🔧 In a real scenario, you would see:
   📋 Listed current notebooks
   🗑️ Deleted demo_notebook.ipynb
   🗑️ Deleted examples/tutorial.ipynb
   📁 Removed empty examples/ directory
   ✅ Cleanup completed successfully

🎯 Content management operations complete!
